In [3]:
import pandas as pd

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"


In [7]:
df_01 = pd.read_csv("Phishing_Email_Data/CEAS_08.csv")
df_02 = pd.read_csv("Phishing_Email_Data/Enron.csv")
df_03 = pd.read_csv("Phishing_Email_Data/Ling.csv")
df_04 = pd.read_csv("Phishing_Email_Data/Nazario.csv")
df_05 = pd.read_csv("Phishing_Email_Data/Nigerian_Fraud.csv")
df_06 = pd.read_csv("Phishing_Email_Data/phishing_email.csv")
df_07 = pd.read_csv("Phishing_Email_Data/SpamAssasin.csv")


In [8]:
df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06, df_07], ignore_index=True)

In [9]:
# Combine subject and body into a single text field per email
df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")
# Filter out emails that are empty or too short to be meaningful
df = df[df["text"].str.strip().str.len() > 20] 


In [10]:
df.head()

,sender,receiver,date,subject,body,label,urls,text_combined,text
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1.0,NaN,"Never agree to be a loser Buck up, your troubl..."
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1.0,NaN,Befriend Jenna Jameson \nUpgrade your sex and ...
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1.0,NaN,CNN.com Daily Top 10 >+=+=+=+=+=+=+=+=+=+=+=+=...
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1.0,NaN,Re: svn commit: r619753 - in /spamassassin/tru...
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1.0,NaN,SpecialPricesPharmMoreinfo \nWelcomeFastShippi...


In [11]:
df["label"].value_counts()

label
1    42882
0    39592
Name: count, dtype: int64

In [35]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

# Specify UTF-8 encoding (most common for text files)
loader = TextLoader("security_guidelines.txt", encoding="utf-8")  
security_docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
security_chunks = splitter.split_documents(security_docs)

In [38]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import os

# Create embeddings model — transforms text chunks into vectors
embedding = OpenAIEmbeddings()

# Build FAISS vectorstore from your chunked security guideline documents
vectorstore_guidelines = FAISS.from_documents(security_chunks, embedding)

# Define a separate folder path for saving security guidelines index and metadata
GUIDELINES_INDEX_PATH = "security_faiss_index"

# Make sure the directory exists
os.makedirs(GUIDELINES_INDEX_PATH, exist_ok=True)

# Save the security guidelines vectorstore locally (index + metadata)
vectorstore_guidelines.save_local(GUIDELINES_INDEX_PATH)


In [ ]:
from langchain.schema import Document

# Create a list of Document objects from the DataFrame `df`
documents = [
    Document(
        # Use the combined email text from the 'text' column as the main content
        page_content=row['text'],  
        
        # Add metadata to each document indicating whether it's phishing or not,
        # based on the 'label' column (1 means phishing, otherwise not phishing)
        metadata={"label": "Phishing" if row['label'] == 1 else "Not Phishing"}
    )
    # Loop through each row of the DataFrame using `iterrows()`
    for _, row in df.iterrows()
]


In [ ]:
import os
import json
import time
from tqdm import tqdm
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import tiktoken

# Setup
embedding_model = OpenAIEmbeddings()
tokenizer = tiktoken.encoding_for_model("text-embedding-ada-002")

MAX_TOKENS_PER_BATCH = 290_000  # keep below 300k limit for batch total tokens
MAX_TOKENS_PER_DOC = 290_000    # max tokens allowed per single document to embed
SLEEP_TIME = 1.5  # rate limit buffer
INDEX_PATH = "faiss_index"
PROGRESS_LOG = os.path.join(INDEX_PATH, "embedding_progress.json")

# Make sure the index folder exists
os.makedirs(INDEX_PATH, exist_ok=True)

# Load your documents (list of Document objects)
# documents = [ ... your docs from DataFrame as before ...]

def num_tokens(text):
    return len(tokenizer.encode(text))

# Load progress log or initialize it
if os.path.exists(PROGRESS_LOG):
    with open(PROGRESS_LOG, "r") as f:
        progress_data = json.load(f)
        start_idx = progress_data.get("last_index", 0) + 1
        print(f"Resuming from document index: {start_idx}")
else:
    start_idx = 0

vectorstore = None
current_batch = []
current_tokens = 0

# If index exists already, load it
faiss_index_file = os.path.join(INDEX_PATH, "index.faiss")
faiss_metadata_file = os.path.join(INDEX_PATH, "index.pkl")
if os.path.exists(faiss_index_file) and os.path.exists(faiss_metadata_file):
    vectorstore = FAISS.load_local(INDEX_PATH, embedding_model, allow_dangerous_deserialization=True)
    print("Loaded existing FAISS index from disk.")
else:
    print("No existing index found, starting fresh.")

for i in tqdm(range(start_idx, len(documents)), desc="Embedding batches"):
    doc = documents[i]
    doc_tokens = num_tokens(doc.page_content)

    # Skip docs that are too large to embed in one request
    if doc_tokens > MAX_TOKENS_PER_DOC:
        print(f"Skipping document {i} with {doc_tokens} tokens — too large to embed.")
        with open(PROGRESS_LOG, "w") as f:
            json.dump({"last_index": i}, f)
        continue

    # If adding this doc would exceed batch token limit, process current batch
    if current_tokens + doc_tokens > MAX_TOKENS_PER_BATCH and current_batch:
        texts = [d.page_content for d in current_batch]
        metadatas = [d.metadata for d in current_batch]

        batch_index = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
        if vectorstore is None:
            vectorstore = batch_index
        else:
            vectorstore.merge_from(batch_index)

        # Save index and progress log after batch embedding
        vectorstore.save_local(INDEX_PATH)
        with open(PROGRESS_LOG, "w") as f:
            json.dump({"last_index": i - 1}, f)

        time.sleep(SLEEP_TIME)
        current_batch = []
        current_tokens = 0

    # Add current doc to batch
    current_batch.append(doc)
    current_tokens += doc_tokens

# Process any remaining documents after loop ends
if current_batch:
    texts = [d.page_content for d in current_batch]
    metadatas = [d.metadata for d in current_batch]

    batch_index = FAISS.from_texts(texts, embedding_model, metadatas=metadatas)
    if vectorstore is None:
        vectorstore = batch_index
    else:
        vectorstore.merge_from(batch_index)

    vectorstore.save_local(INDEX_PATH)
    with open(PROGRESS_LOG, "w") as f:
        json.dump({"last_index": len(documents) - 1}, f)

print("Embedding complete and saved.")


Resuming from document index: 71769
Loaded existing FAISS index from disk.


Embedding batches:   4%|▍         | 430/10705 [00:01<00:24, 423.14it/s]

Skipping document 71769 with 2955173 tokens — too large to embed.


Embedding batches:   6%|▋         | 673/10705 [00:01<00:19, 515.39it/s]

Skipping document 72302 with 380571 tokens — too large to embed.


Embedding batches: 100%|██████████| 10705/10705 [12:16<00:00, 14.53it/s]


Embedding complete and saved.


In [39]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI 

# Load the GPT-3.5 Turbo model to use as our language model (LLM)
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo") #Controls randomness: 0 = fully deterministic outputs

# Tells LangChain to retrieve based on semantic closeness and Fetches top 3 most relevant chunks per query
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3}) 

# For security guidelines
retriever_guidelines = vectorstore_guidelines.as_retriever(search_type="similarity", search_kwargs={"k": 3})


# Build the QA pipeline: LLM + semantic retriever → answer generation
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

qa_chain_guidelines = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_guidelines,
    return_source_documents=True
)

In [41]:
# Email classification function
def classify_email(email_text):
    prompt = (
        "Evaluate whether the following email is a phishing attempt. "
        "Reply with 'Phishing' or 'Not Phishing', and explain your reasoning based on cues like urgency, suspicious links, or sender identity.\n\n"
        f"{email_text}"
    )
    result = llm.predict(prompt)
    return result.strip()

# email
email = """
Subject: Urgent Password Reset

Dear User,

Your account has been flagged. Please reset your password immediately using the link below or your account will be suspended.

[Reset Now]

Regards,
Admin
"""

classification = classify_email(email)
# Get advice from emails vectorstore QA chain
advice_from_emails = qa_chain.invoke(
    f"What are the recommended actions according to cybersecurity best practices if I receive an email like this:\n\n{email}"
)

# Get advice from security guidelines vectorstore QA chain
advice_from_guidelines = qa_chain_guidelines.invoke(
    f"What are the recommended actions according to cybersecurity best practices if I receive an email like this:\n\n{email}"
)

print("Classification:", classification)
print("\nAdvice from similar emails:\n", advice_from_emails['result'])
print("\nAdvice from security guidelines:\n", advice_from_guidelines['result'])



Classification: Phishing

Reasoning: The email creates a sense of urgency by claiming the account has been flagged and will be suspended if the password is not reset immediately. The sender is simply identified as "Admin" without any specific information about the company or organization. The link provided could potentially lead to a phishing website designed to steal login credentials.

Advice from similar emails:
 Based on the information provided, the email you received exhibits characteristics of a phishing attempt. Here are the recommended actions according to cybersecurity best practices:

1. **Do not click the link:** Avoid clicking on any links in the email, especially if it is urging you to take immediate action.

2. **Verify sender's email address:** Check the sender's email address carefully for any slight variations or spoofing that may indicate it is not from a legitimate source.

3. **Hover over the link:** Hover over the link provided in the email to preview the URL with

In [42]:
for i, doc in enumerate(advice['source_documents'], 1):
    print(f"--- Email chunk #{i} ---")
    print("Content:")
    print(doc.page_content if doc.page_content else "[No content]")
    print("Metadata:")
    print(doc.metadata)
    print("\n")


--- Email chunk #1 ---
Content:
* * your account may suspend * * dear paypal member ,
you have received this email as part of a verified paypal campaign meant to
increase security for your credit card against online credit card fraud .
verified paypal has detected that you have been using this email address for
online purchases and in order to protect yourself against online credit card
fraud we would like to introduce you to a new system that will protect you
against frauds .
you can associate your email address to your credit card and receive a
password that you will use for any online purchase . also you will be notified
by verified paypal when an online purchase is made .
follow the below and go to verified paypal . you can join the verified paypal
system or learn more about this .

Metadata:
{'label': 'Phishing'}


--- Email chunk #2 ---
Content:
dear customer your details have been compromised dear customer :
recently there have been a large number of cyber attacks pointing our d